In [1]:
import csv
        
reader = csv.reader(open("results.csv", 'rU'), dialect='excel')
# reader = pd.read_csv("results.csv", encoding='utf-8')
authors = []
dates = []
titles = []
urls = []
scores = []
platforms = []
review_text = []

for col in reader:
    dates.append(col[0])
    titles.append(col[1])
    urls.append(col[2])
    platforms.append(col[3])
    review_text.append(col[4])
    authors.append(col[6])
    scores.append(col[8])


In [2]:
import pandas as pd
import numpy as np

index = range(len(scores))
# columns = ['authors','dates','titles','urls','scores','platforms','reviews']
columns = ['authors','titles','urls','scores','platforms','reviews']

review_data = pd.DataFrame(index=index, columns=columns)
review_data['authors'] = authors
# review_data['dates'] = dates
review_data['titles'] = titles
review_data['urls'] = urls
review_data['scores'] = scores
review_data['platforms'] = platforms
review_data['reviews'] = review_text

review_data.ix[:1, :7]

,authors,titles,urls,scores,platforms,reviews
0,Jeff Gerstmann,SingStar,http://www.giantbomb.com/api/review/1900-1/,80,PlayStation 3,All these karaoke video games have got it all ...
1,Jeff Gerstmann,Haze,http://www.giantbomb.com/api/review/1900-2/,40,PlayStation 3,Haze attempts to deal with the concept of wart...


In [3]:
import pandas as pd
import numpy as np

# index = range(415)
columns = ['authors','reviews']

author_review = pd.DataFrame(index=index, columns=columns)
author_review['authors'] = authors
author_review['reviews'] = review_text

author_review.ix[:1, :2]

,authors,reviews
0,Jeff Gerstmann,All these karaoke video games have got it all ...
1,Jeff Gerstmann,Haze attempts to deal with the concept of wart...


In [4]:
author_review['reviews'] = author_review[['authors','reviews']].groupby(['authors'])['reviews'].transform(lambda x: ','.join(x))
author_review = author_review[['authors','reviews']].drop_duplicates()
author_review = author_review.reset_index()

In [5]:
print author_review

     index            authors  \
0        0     Jeff Gerstmann   
1        2     Brad Shoemaker   
2        6         Ryan Davis   
3       34    Vinny Caravella   
4       87       Andy McCurdy   
5      116          Dave Lang   
6      201       Drew Scanlon   
7      289       Alex Navarro   
8      389      Matthew Rorie   
9      408     Patrick Klepek   
10     630        Dan Ryckert   
11     659      Kallie Plagge   
12     660      Dan Stapleton   
13     661    Miranda Sanchez   
14     662     Vince Ingenito   
15     664         Mitch Dyer   
16     665        Marty Sliva   
17     666       Lucy O'Brien   
18     667    Meghan Sullivan   
19     668       Seth G. Macy   
20     669        Luke Reilly   
21     670           Cam Shea   
22     673        Dave Rudden   
23     675     Ryan McCaffrey   
24     677      Mikel Reparaz   
25     679       Leif Johnson   
26     680         Jose Otero   
27     681          Rob Zacny   
28     685      Mike Mitchell   
29     689

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import unicodedata
from nltk import stem
import string
# import sys
# reload(sys)
# sys.setdefaultencoding("utf-8")


stopset = set(stopwords.words('english'))
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
special_char = ['~~','``',"''", '--',"'ll","n't","'re","'s","'d","term"]

names=author_review['authors'].unique().tolist()
text=author_review['reviews'].unique().tolist()

vocab = []

for i in range(len(names)):
    with open('/Users/feismacbookpro/Desktop/INFX575/Final Project/author_reviews3/'+names[i]+'.txt', 'w') as f:
        tokens = word_tokenize(str(text[i]).decode("ISO-8859-1"))
        
        # remove stop words, stem words, digits, single letters, and special characters
        # and get cleaned unigrams
        uni_cleaned = ''
        vocab_tmp = []
        stemmer=stem.PorterStemmer()
        for w in tokens:
            w = w.lower()
            if w not in stopset:
                tmp = stemmer.stem(w)
                tmp = unicodedata.normalize('NFKD', tmp).encode('ascii','ignore')
                if tmp not in string.punctuation:
                    if not tmp.isdigit():
                        if tmp not in letters:
                            if tmp not in special_char:                               
                                vocab_tmp.append(tmp)
        for u in vocab_tmp:
            if (vocab_tmp.count(u) >= 10):
                uni_cleaned = uni_cleaned + u + ' '
                vocab.append(u)
                
        f.write(uni_cleaned)
        f.close()

In [7]:
vocab = set(list(vocab))
print len(vocab)
vocab_new = []
# vocab for all 40 combined reviews: vocab_new
for v in vocab:
    vocab_new.append(v)
print len(vocab_new)

3349
3349


In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
import os

# define a function that creates a term document matrix as pandas dataframe
# **kwargs indicates the arguments of CountVectorizer that can be passed
def fn_tdm_df(docs, xColNames = None, **kwargs):
    #initialize the  vectorizer
    vectorizer = CountVectorizer(**kwargs)
    x1 = vectorizer.fit_transform(docs)
    #create dataFrame
    df = pd.DataFrame(x1.toarray().transpose(), index = vectorizer.get_feature_names())
    if xColNames is not None:
        df.columns = xColNames
    return df

# define a function that creates corpus from a dictionary
def fn_CorpusFromDIR(xDIR):
    Res = dict(docs = [open(os.path.join(xDIR,f)).read() for f in os.listdir(xDIR)],
               ColNames = map(lambda x: x[0:], os.listdir(xDIR)))
    return Res
 
# provide directory where files (to be used to generate the matrix) are saved
DIR = '/Users/feismacbookpro/Desktop/INFX575/Final Project/author_reviews2/'

# call functions above to create a term document matrix as a pandas dataframe
d = fn_tdm_df(docs = fn_CorpusFromDIR(DIR)['docs'],
          xColNames = fn_CorpusFromDIR(DIR)['ColNames'], 
          stop_words='english', charset_error = 'replace') 

# arrange the result to certain format for LDA use
# d = d.drop(['.DS_Store'], axis=1)
d_copy = d.transpose()
print np.array(d_copy)

[[17  0 55 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 ..., 
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]]


/Users/feismacbookpro/anaconda/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:654: DeprecationWarning: The charset_error parameter is deprecated as of version 0.14 and will be removed in 0.16. Use decode_error instead.
  DeprecationWarning)


In [9]:
print (np.array(d_copy).shape)

(132, 2208)


In [10]:
import numpy as np
import lda

# specify parameters for LDA model -  number of topics, number of iterations, number of words per topic)
# and fit the model with generated term document matrix
model = lda.LDA(n_topics=40, n_iter=1000, random_state=1)
model.fit(np.array(d_copy))

topic_word = model.topic_word_
n_top_words = 10

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab_new)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: classic interview theori coolest hill express bourn creatur saint
Topic 1: classic theori definit interview separ neverdead voiceov bourn profil
Topic 2: classic interview coolest hill tron phantasma massiv express maw
Topic 3: classic interview filter bourn hill voiceov theori creatur 'm
Topic 4: classic henk bourn monochroma creatur purpos warrior though crimsonland
Topic 5: interview neverdead tron theori classic coolest hill filter dedic
Topic 6: interview express spread tron bolt filter fist theori card
Topic 7: say sunken though dedic develop girl rise scenario mice
Topic 8: say soundtrack interview maw first classic hill dedic good
Topic 9: classic filter theori human hill interview creatur express bourn
Topic 10: interview henk case say hill attempt fault girl scenario
Topic 11: neverdead extract rocksmith vessel section loos hardli numer firebal
Topic 12: classic read bourn henk hill ive solo one hardlin
Topic 13: classic hill coolest tron microphon rise interview cas